In [1]:
#code to load a dataset and save the tensors in a new directory (image to tensor)
from torch.utils.data import Dataset
import os
import torch
from PIL import Image
from transformers import CLIPModel, CLIPProcessor

wordlist_path = "wordlist/selected_words_point001.txt" 
image_dir = "Images/coco_val2017/val2017"

torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

new_dir_path = "Images/coco_val2017/new_tensors"

os.makedirs(new_dir_path, exist_ok=True)
print('made new directory')

with open(wordlist_path, 'r') as f:
    wordlist = [line.strip() for line in f if line.strip()]
print(f"random test word: {wordlist[69]}")
image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('.jpg', '.png'))]
images = [Image.open(img).convert("RGB") for img in image_paths]

for i, image in enumerate(images):
    print(f"Progress: {i+1}/{len(images)}", end="\r")
    #get clip output
    clip_input = processor(text=wordlist, images=image, return_tensors='pt', padding=True)
    clip_input = {key: val.to(device) for key, val in clip_input.items()}
    clip_output = clip_model(**clip_input)
    logits = clip_output.logits_per_image
    probs = logits.softmax(dim=1)
    #print(probs)
    #save in new directory
    filename = new_dir_path + f"/tensor_{i}.pt"
    torch.save(probs, filename)
print('done')

/home/jans26/.conda/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


made new directory
random test word: post
doneress: 5000/5000
